In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
from pathlib import Path
from datasets import load_from_disk
from sentence_transformers import SentenceTransformer
import numpy as np
import srsly
import hnswlib as hb
from src.data.datamodule import DataModule

In [5]:
model_name = "all-mpnet-base-v2" # "all-MiniLM-L6-v2"
sentence_encoder = SentenceTransformer(model_name)

In [6]:
data_path = Path("../data/prepared/civil_comments/")
ds_dict = load_from_disk(data_path)
train_ds = ds_dict["train"] #.select(range(1000))
train_df = train_ds.to_pandas()

/home/pl487/.conda/envs/allset/lib/python3.9/site-packages/datasets/dataset_dict.py:1241: FutureWarning: 'fs' was is deprecated in favor of 'storage_options' in version 2.8.0 and will be removed in 3.0.0.
You can remove this warning by passing 'storage_options=fs.storage_options' instead.
  warnings.warn(


In [7]:
embeddings = np.load("../data/prepared/civil_comments/civil_comments_index.npy")

In [8]:
dm = DataModule.from_dataset_dict(ds_dict)

In [9]:
meta = srsly.read_yaml(data_path / "metadata.yaml")

In [10]:
dm.load_index(meta["hnsw_index_path"], meta["embedding_dim"])

In [12]:
indices, distances = dm.search_index(embeddings[0, :], 10)

In [15]:
indices.flatten()

array([5248293, 5181976, 1034577,  443060, 6212178, 1035512, 6013837,
        870843,  759602, 6295424], dtype=uint64)

In [ ]:
embeddings = sentence_encoder.encode(train_df["text"].tolist(), show_progress_bar=True, batch_size=512, device="cuda")

In [ ]:
np.save("../data/processed/civil_comments/civil_comments_index.npy", embeddings, fix_imports=False)

In [ ]:
meta = srsly.read_yaml("../data/processed/civil_comments/metadata.yaml")

In [ ]:
meta["embedding_model"] = model_name

In [ ]:
srsly.write_yaml("../data/processed/civil_comments/metadata.yaml", meta)

In [23]:
dm.index.mark_deleted(5257240)

RuntimeError: The requested to delete element is already deleted

In [30]:
items = np.array(dm.index.get_ids_list()[:10])
dm.index.get_items(items)

RuntimeError: Label not found

In [25]:
dm.index.get_ids_list()[:10]

[5257240,
 5702599,
 5178523,
 5116155,
 603749,
 254498,
 5005274,
 815497,
 5593338,
 335127]

In [ ]:
p = hb.Index(space="cosine", dim=embeddings.shape[1])
p.set_ef(200)
p.init_index(max_elements=embeddings.shape[0], M=64, ef_construction=200)
p.add_items(embeddings, np.arange(embeddings.shape[0]))

In [ ]:
query = embeddings[:2, :]

In [ ]:
indices, scores = p.knn_query(query, k=10)

In [ ]:
p.save_index("../data/processed/civil_comments/civil_comments_index.bin")

In [ ]:
p = hb.Index(space="cosine", dim=embeddings.shape[1])
p.load_index("../data/processed/civil_comments/civil_comments_index.bin")

In [ ]:
ids, distances = p.knn_query(embeddings[100, :], k=10)

In [ ]:
train_df.loc[
    train_df["unique_id"].isin(ids[0, 1:]), 
    "text"
].tolist()

In [ ]:
from src.data.datamodule import DataModule

In [ ]:
dm = DataModule.from_dataset_dict(ds_dict)